In [ ]:
# Install dependencies

!apt update -y -qq
!apt install -y -qq curl lshw libcairo2-dev pkg-config python3-dev
!pip install gradio
!pip install -qU langchain langchain_community
!pip install -qU langchain_chroma
!pip install -qU langchain_ollama
!pip install -qU beautifulsoup4
!pip install -qU pypdf

In [2]:
# Import Libs

import os
import glob
import subprocess
from dotenv import load_dotenv
import gradio as gr
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [ ]:
# Run Ollama

sub = subprocess.Popen("ollama serve", shell=True, stdout=subprocess.PIPE)
!ollama pull nomic-embed-text

In [8]:
# Fetch Knowledge Base and convert it to chunks of data

folders = glob.glob("/kaggle/input/rag-kb/RAG Knowledge Base/*")

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.txt", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])

text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=400)
chunks = text_splitter.split_documents(documents)

In [9]:
# Create embedding vector and store it by chroma

DB_NAME = "vector_db"

embeddings = OllamaEmbeddings(model="nomic-embed-text")

# Delete if already exists
if os.path.exists(DB_NAME):
    Chroma(persist_directory=DB_NAME, embedding_function=embeddings).delete_collection()

# Create vectorstore
vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=DB_NAME)

In [ ]:
# Setup Langchain

MODEL = "gemma3:12b"
!ollama pull gemma3:12b
llm = ChatOllama(temperature=0.7, model=MODEL)

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [ ]:
# Lunch chat interface

def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)